# Projet mathématiques-informatique



## Algorithme A* et taquin

### Importation des librairies nécessaires

In [8]:
import numpy as np
import math as m
import types
import pandas as pd
import time
from itertools import permutations
import random
import matplotlib.pyplot as plt



plt.rcParams["figure.figsize"] = [16, 9]
plt.rcParams["figure.dpi"] = 300
plt.rcParams['font.size'] = 16

### Définition du dictionnaire des directions

In [9]:
# Dictionnaire des directions possibles pour la case vide ou "0" ici :
# Soit i, j les indices de la case vide du puzzle
# i + 1 revient à descendre d'une case
# j + 1 revient à bouger vers la droite
# i - 1 revient à monter d'une case
# j - 1 revient à bouger vers la gauche
direct_dico = {
    (1, 0): "BAS",
    (0, 1): "DROITE",
    (-1, 0): "HAUT",
    (0, -1): "GAUCHE",
    }

### Définition des fonctions nécessaires à la représentation du taquin

In [10]:
def cherche_indices(puzzle: str, numero: str, *args) -> (int, int):
    """
    

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    numero : str
        DESCRIPTION.
    *args : TYPE
        DESCRIPTION.

    Returns
    -------
    (int, int)
        DESCRIPTION.

    """
    indice = puzzle.find(numero)
    i = indice // 3
    j = indice - i * 3
    
    
    return i, j


def directions(puzzle: str, *args) -> list:
    """
    Cherche les directions possibles

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    *args : TYPE
        DESCRIPTION.

    Returns
    -------
    list
        DESCRIPTION.

    """
    i, j = cherche_indices(puzzle, "0")
    
    
    # On cherche les directions dans lesquelles 0 peut aller
    direction = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    resultat = []
    
    
    for di in direction:
        if not ((i + di[0] < 0) or (i + di[0] > 2) or (j + di[1] < 0) or (j + di[1] > 2)):
            resultat.append(direct_dico[di])
    
    return resultat

### Définition des différentes fonctions génératrices d'heuristique

In [11]:
def heuristique_manhattan(puzzle: str, fin: str) -> int:
    """
    Somme des distances de Manhattan de chaque case entre le puzzle actuel et le puzzle final

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    fin : str
        DESCRIPTION.

    Returns
    -------
    int
        DESCRIPTION.

    """
    poids_h = 0
    
    
    for case in range(len(puzzle)):
        i, j = cherche_indices(puzzle, str(case))
        x, y = cherche_indices(fin, str(case))
        
        h = abs(i - x) + abs(j - y)
        poids_h += h
    
    
    return poids_h


def heuristique_malplace(puzzle: str, fin: str) -> int:
    """
    Somme des cases mal placées entre le puzzle actuel et le puzzle final

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    fin : str
        DESCRIPTION.

    Returns
    -------
    int
        DESCRIPTION.

    """
    poids_h = 0
    
    
    for case in range(len(puzzle)):
        i, j = cherche_indices(puzzle, str(case))
        x, y = cherche_indices(fin, str(case))
        
        if not ((i == x) or (j == y)):
            poids_h += 1
    
    
    return poids_h


def heuristique_zeros(puzzle: str, fin: str) -> int:
    """
    Heuristique nulle

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    fin : str
        DESCRIPTION.

    Returns
    -------
    int
        DESCRIPTION.

    """
    poids_h = 0
    
    
    return poids_h


def heuristique_inverse_malplace(puzzle: str, fin: str) -> int:
    """
    Cinquante moins la somme des cases mal placées entre le puzzle actuel et le puzzle final

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    fin : str
        DESCRIPTION.

    Returns
    -------
    int
        DESCRIPTION.

    """
    poids_h = 0
    
    
    for case in range(len(puzzle)):
        i, j = cherche_indices(puzzle, str(case))
        x, y = cherche_indices(fin, str(case))
        
        if not ((i == x) or (j == y)):
            poids_h += 1
    
    
    return 50 - poids_h

### Fonctions utilitaires

In [12]:
def etats(puzzle: str, directions: list, *args) -> list:
    """
    Génère les états futurs (chemins possibles)

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    directions : list
        DESCRIPTION.
    *args : TYPE
        DESCRIPTION.

    Returns
    -------
    list
        DESCRIPTION.

    """
    # On cherche les coordonnées de 0
    i, j = cherche_indices(puzzle, "0")
    
    
    # On cherche maintenant les coups possibles
    deplacements = []
    for di in directions:
        deplacements.append((i + list(direct_dico.keys())[list(direct_dico.values()).index(di)][0], j + list(direct_dico.keys())[list(direct_dico.values()).index(di)][1]))
    
    
    # On forme les nouveaux états possibles
    liste_etats = []
    # resultat = []
    for mouvement in deplacements:
        # On transforme notre puzzle sous forme de chaîne de caractères en liste
        numeros = list(puzzle)
        
        # On "aplatit la matrice" pour passer de la notation (i, j) à la notation k avec k = i * 3 + j
        indice_depart = i * 3 + j
        indice_arrivee = mouvement[0] * 3 + mouvement[1]
        
        # Echange de position entre le départ et l'arrivée
        numeros[indice_depart], numeros[indice_arrivee] = numeros[indice_arrivee], numeros[indice_depart]
        
        # On transforme notre puzzle sous forme de liste en chaîne de caractères
        liste_etats.append(''.join(numeros))
        
        
    resultat = []
    for element in range(len(liste_etats)):
        resultat.append([liste_etats[element], directions[element]])
    
    
    return resultat


def affichage(cheminement: list, *args):
    """
    Affiche le chemin de la victoire

    Parameters
    ----------
    cheminement : list
        DESCRIPTION.
    *args : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    for i in range(len(cheminement)):
        puzzle = cheminement[i][0]
        phrase = cheminement[i][1]
        g = i
        h = cheminement[i][2]
        f = g + h
        
        
        # Le poids de chaque étape est équivalent d'où g est égal
        # au nombre d'étapes nécessaires pour atteindre l'état présent
        print(f"Étape {i}: {phrase} - g = {g}, h = {h} soit f = {f}")
        
        
        for j in range(0, 9, 3):
            print(f"| {puzzle[j]} | {puzzle[j+1]} | {puzzle[j+2]} |")


def resoluble(puzzle: str, N: int, print_ON: bool = False) -> bool:
    """
    Évalue si le taquin est résoluble en fonction de la parité des côtés du puzzle, le nombre de case séparant la case vide du bas du puzzle
    ainsi que le nombre d'inversions c'est-à-dire le nombre de fois où un chiffre est placé avant les autres chiffres plus élevés que lui
    Soit 1, 2, 3, 4, 5, 6, 8, 7 un puzzle "aplati", on ne tient pas compte de la case vide:
    alors il y a 1 inversion car le 8 est placé avant le 7

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    N : int
        DESCRIPTION.
    print_ON : bool, optional
        DESCRIPTION. The default is False.

    Returns
    -------
    bool
        DESCRIPTION.

    """
    tableau = list(puzzle)
    
    
    compteur = 0
    for i in range(N * N - 1):
        for j in range(i + 1, N * N):
            if (tableau[j] and tableau[i] and tableau[i] > tableau[j]):
                compteur += 1
    
    
    position = N - cherche_indices(puzzle, "0")[0]
    
    
    # Si le puzzle a des côtés impairs, retourne si le nombre d'inversions
    # est pair
    if (N % 2 == 1):
        boolean = (compteur % 2 == 0)
        
        if print_ON:
            print(boolean)
        
        return boolean
    
    
    # Si le puzzle a des côtés pairs, retourne si le nombre d'inversions
    # est pair lorsque la case vide est à un nombre de cases impair du bas
    # Sinon retourne si le nombre d'inversions est impair lorsque
    # la case vide est à un nombre de cases pair du bas
    else:
        if (position % 2 == 1):
            boolean = (compteur % 2 == 0)
            
            if print_ON:
                print(boolean)
            
            return boolean
        else:
            boolean = (compteur % 2 == 1)
            
            if print_ON:
                print(boolean)
            
            return boolean

### Algorithme A*

In [13]:
def a_etoile(debut: str, fin: str, heuristique: types.FunctionType, liste_FERME: bool = True, *args) -> (list, int):
    """
    Implémentation de l'algorithme A*, si liste_FERME est mis sur True alors un nœud ne pourra pas être étendu plusieurs fois
    car on utilise la liste FERME
    Si on veut étendre plusieurs fois le même nœud, liste_FERME doit être False

    Parameters
    ----------
    debut : str
        DESCRIPTION.
    fin : str
        DESCRIPTION.
    heuristique : types.FunctionType
        DESCRIPTION.
    liste_FERME : bool, optional
        DESCRIPTION. The default is True.
    *args : TYPE
        DESCRIPTION.

    Returns
    -------
    (list, int)
        DESCRIPTION.

    """
    if resoluble(debut, 3):
        OUVERT = []
        
        
        # Implémentation de l'algorithme A* où l'on ne peut pas étendre plusieurs fois un nœud
        if liste_FERME:
            FERME = {
                debut: (1, heuristique(debut, fin)),
                }
        
        
        iteration = 0
        
        
        OUVERT.append([[debut, "DÉPART", heuristique(debut, fin), 1]])
        
        
        
        while OUVERT:
            # Montre le nombre d'itérations
            print(f"\rIterations: {iteration}", end="\r")
            
            iteration += 1
            
            
            # L'algorithme choisit dans OUVERT le nœud ayant le plus faible score f
            etat_present = OUVERT.pop(0)
            puzzle_present = etat_present[-1][0]
            
            
            # A-t-on gagné ?
            if puzzle_present == fin:
                etat = etat_present
                
                print("")
                print("FIN")
                
                
                return etat, iteration
            
            
            # On estime g, le coût
            g = len(etat_present)
            
            
            # On génère les déplacements possibles de la case vide ou "0" ici
            etat_suivant = etats(puzzle_present, directions(puzzle_present))
            
            
            # On compare les états possibles
            for etat in etat_suivant:
                etat_puzzle = etat[0]
                g_etat = g + 1
                
                # Implémentation de l'algorithme A* où l'on ne peut pas étendre plusieurs fois un nœud
                if liste_FERME:
                    
                    
                    # On vérifie si l'on a déjà développé le nœud (puzzle) auparavant
                    if (etat_puzzle in FERME) or (etat_puzzle in [x[-1][0] for x in OUVERT]):
                        
                        
                        # Si une occurrence est trouvée dans OUVERT et que le score f de l'occurrence est plus faible que le score f
                        # du nœud étudié actuellement, ne rien faire
                        # Sinon l'occurrence est supprimée de OUVERT et on rajoute le nœud actuel à OUVERT
                        if (etat_puzzle in [x[-1][0] for x in OUVERT]):
                            liste_occurrence = [x[-1][0] for x in OUVERT]
                            indice_occurrence = liste_occurrence.index(etat_puzzle)
                            occurrence = OUVERT[indice_occurrence]
                            if occurrence[-1][2] + occurrence[-1][3] < heuristique(etat_puzzle, fin) + g_etat:
                                continue
                            else:
                                OUVERT.pop(indice_occurrence)
                                ajout_etat = etat_present.copy()
                                etat.append(heuristique(etat_puzzle, fin))
                                etat.append(g_etat)
                                ajout_etat.append(etat)
                                OUVERT.append(ajout_etat)
                        
                        
                        # Si une occurrence est trouvée dans FERME et que le score f de l'occurrence est plus faible que le score f
                        # du nœud étudié actuellement, ne rien faire
                        # Sinon l'occurrence est supprimée de FERME et on rajoute le nœud actuel à OUVERT
                        if (etat_puzzle in FERME):
                            if FERME[etat_puzzle][1] + FERME[etat_puzzle][0] < heuristique(etat_puzzle, fin) + g_etat:
                                continue
                            else:
                                FERME.pop(etat_puzzle)
                                ajout_etat = etat_present.copy()
                                etat.append(heuristique(etat_puzzle, fin))
                                etat.append(g_etat)
                                ajout_etat.append(etat)
                                OUVERT.append(ajout_etat)
                   
                    
                   
                    else:
                         ajout_etat = etat_present.copy()
                         etat.append(heuristique(etat_puzzle, fin))
                         etat.append(g_etat)
                         ajout_etat.append(etat)
                         OUVERT.append(ajout_etat)
                
                
                
                else:
                    
                    
                    # On vérifie si l'on a déjà développé le nœud (puzzle) auparavant
                    if (etat_puzzle in [x[-1][0] for x in OUVERT]):
                        
                        
                        # Si une occurrence est trouvée dans OUVERT et que le score f de l'occurrence est plus faible que le score f
                        # du nœud étudié actuellement, ne rien faire
                        # Sinon l'occurrence est supprimée de OUVERT et on rajoute le nœud actuel à OUVERT
                        liste_occurrence = [x[-1][0] for x in OUVERT]
                        indice_occurrence = liste_occurrence.index(etat_puzzle)
                        occurrence = OUVERT[indice_occurrence]
                        if occurrence[-1][2] + occurrence[-1][3] < heuristique(etat_puzzle, fin) + g_etat:
                            continue
                        else:
                            OUVERT.pop(indice_occurrence)
                            ajout_etat = etat_present.copy()
                            etat.append(heuristique(etat_puzzle, fin))
                            etat.append(g_etat)
                            ajout_etat.append(etat)
                            OUVERT.append(ajout_etat)
                   
                    
                   
                    else:
                         ajout_etat = etat_present.copy()
                         etat.append(heuristique(etat_puzzle, fin))
                         etat.append(g_etat)
                         ajout_etat.append(etat)
                         OUVERT.append(ajout_etat)
                
                
            # Tri des chemins en fonction du score g (longueur du chemin) et score h (heuristique de l'algorithme choisie)
            OUVERT = sorted(OUVERT, key = lambda x: x[-1][-1] + x[-1][-2])
            
            
            # On ajoute le nœud ayant le plus faible score f dans la liste FERME
            etat_suivant = OUVERT[0]
            puzzle_present = etat_present[-1][0]
            
            
            # Implémentation de l'algorithme A* où l'on ne peut pas étendre plusieurs fois un nœud
            if liste_FERME:
                FERME[puzzle_present] = g_etat, heuristique(puzzle_present, fin)
            
            
        pass
    
    
    
    else:
        print("Le taquin n'est pas résoluble...")

### Exemples



On procède maintenant aux différents essais.

In [14]:
def combientaquin(nbr: int) -> list:
    """
    Donne le nombre de taquins résolubles qu'on veut

    Parameters
    ----------
    nbr : int
        DESCRIPTION.

    Returns
    -------
    list
        DESCRIPTION.

    """
    perms = [''.join(p) for p in permutations('1234567890')]
    # print(random.sample(list(perms), nbr))
    valide = []
    
    
    while len(valide) < nbr:
        test = random.choice(list(perms))
        if resoluble(test, 3) == True:
            but = "123456780"
            solution, iterations = a_etoile(test, but, heuristique_manhattan, liste_FERME = False)
            valide.append(test, len(solution))
    
    
    return valide


def choix(puzzle: str, fin: str, heuristique: types.FunctionType, FERME: bool, optimalite: int, data: list):
    """
    Fonction finale

    Parameters
    ----------
    puzzle : str
        DESCRIPTION.
    fin : str
        DESCRIPTION.
    heuristique : types.FunctionType
        DESCRIPTION.
    FERME : bool
        DESCRIPTION.
    optimalite : int
        DESCRIPTION.
    data : list
        DESCRIPTION.

    Returns
    -------
    None.

    """
    start_time = time.time()
    data.append({})
    solution, iterations = a_etoile(puzzle, fin, heuristique, liste_FERME = FERME)
    affichage(solution)
    temps_time = time.time() - start_time
    temps = "%s minutes et" % str((time.time() - start_time)//60) + " %s secondes" % str((time.time() - start_time)%60)
    data[-1]["Taquin"] = puzzle
    data[-1]["liste_FERME"] = FERME
    data[-1]["Heuristique"] = heuristique.__name__
    data[-1]["Temps de calcul (s)"] = temps_time
    data[-1]["Temps de calcul"] = temps
    data[-1]["Nœuds visités"] = iterations
    data[-1]["Optimalité de la réponse"] = (len(solution) == optimalite)



# Le tableau sera construit de la manière suivante :
# [{"Taquin": str, "liste_FERME": bool, "Heuristique": types.FunctionType, "Temps de calcul": str, "Nœuds visités": int, "Optimalité de la réponse": bool, ...]
data = []


but = "123456780"



# exemples = combientaquin(5)
# print(exemples)

exemples = [("123805647", 15), ("150734628", 19), ("162743058", 9)]


heuristiques = [heuristique_manhattan, heuristique_malplace, heuristique_zeros]


for exemple in exemples:
    for heuristique in heuristiques:
        choix(exemple[0], but, heuristique, False, exemple[1], data)
        choix(exemple[0], but, heuristique, True, exemple[1], data)


df = pd.DataFrame(data)


histogrammes = ["Temps de calcul (s)", "Nœuds visités"]
for i, histo in enumerate(histogrammes):
    for j, variable in enumerate(set(df["Heuristique"])):
        fig, ax = plt.subplots()
        df[(df["Heuristique"] == variable) & (df["liste_FERME"] == False)].plot(x = 'Taquin', y = histo, kind = 'bar', alpha = 0.5, label = "Sans liste FERME", color = 'b', ax = ax)
        df[(df["Heuristique"] == variable) & (df["liste_FERME"] == True)].plot(x = 'Taquin', y = histo, kind = 'bar', alpha = 0.5, label = "Avec liste FERME", color = 'r', ax = ax)
        plt.ylabel(histo)
        ax.set_title(f'{variable}_{histo}')
        plt.savefig(f'{variable}_{histo}.pdf')
        plt.show()

Iterations: 249
FIN
Étape 0: DÉPART - g = 0, h = 12 soit f = 12
| 1 | 2 | 3 |
| 8 | 0 | 5 |
| 6 | 4 | 7 |
Étape 1: GAUCHE - g = 1, h = 12 soit f = 13
| 1 | 2 | 3 |
| 0 | 8 | 5 |
| 6 | 4 | 7 |
Étape 2: BAS - g = 2, h = 10 soit f = 12
| 1 | 2 | 3 |
| 6 | 8 | 5 |
| 0 | 4 | 7 |
Étape 3: DROITE - g = 3, h = 8 soit f = 11
| 1 | 2 | 3 |
| 6 | 8 | 5 |
| 4 | 0 | 7 |
Étape 4: DROITE - g = 4, h = 6 soit f = 10
| 1 | 2 | 3 |
| 6 | 8 | 5 |
| 4 | 7 | 0 |
Étape 5: HAUT - g = 5, h = 8 soit f = 13
| 1 | 2 | 3 |
| 6 | 8 | 0 |
| 4 | 7 | 5 |
Étape 6: GAUCHE - g = 6, h = 10 soit f = 16
| 1 | 2 | 3 |
| 6 | 0 | 8 |
| 4 | 7 | 5 |
Étape 7: GAUCHE - g = 7, h = 10 soit f = 17
| 1 | 2 | 3 |
| 0 | 6 | 8 |
| 4 | 7 | 5 |
Étape 8: BAS - g = 8, h = 8 soit f = 16
| 1 | 2 | 3 |
| 4 | 6 | 8 |
| 0 | 7 | 5 |
Étape 9: DROITE - g = 9, h = 6 soit f = 15
| 1 | 2 | 3 |
| 4 | 6 | 8 |
| 7 | 0 | 5 |
Étape 10: DROITE - g = 10, h = 4 soit f = 14
| 1 | 2 | 3 |
| 4 | 6 | 8 |
| 7 | 5 | 0 |
Étape 11: HAUT - g = 11, h = 4 soit f = 15
| 1

KeyboardInterrupt: 

In [ ]:
df.style

,Taquin,liste_FERME,Heuristique,Temps de calcul (s),Temps de calcul,Nœuds visités,Optimalité de la réponse
0,123805647,False,heuristique_manhattan,0.040968,0.0 minutes et 0.04096841812133789 secondes,250,True
1,123805647,True,heuristique_manhattan,0.009726,0.0 minutes et 0.009725809097290039 secondes,81,True
2,123805647,False,heuristique_malplace,1.731703,0.0 minutes et 1.7317025661468506 secondes,2166,True
3,123805647,True,heuristique_malplace,0.461642,0.0 minutes et 0.461641788482666 secondes,1178,True
4,123805647,False,heuristique_zeros,25.013105,0.0 minutes et 25.013104915618896 secondes,7768,True
5,123805647,True,heuristique_zeros,8.507867,0.0 minutes et 8.507867336273193 secondes,5510,True
6,150734628,False,heuristique_manhattan,0.113795,0.0 minutes et 0.11379480361938477 secondes,499,True
7,150734628,True,heuristique_manhattan,0.041215,0.0 minutes et 0.041214942932128906 secondes,213,True
8,150734628,False,heuristique_malplace,48.244789,0.0 minutes et 48.24478888511658 secondes,9197,True
9,150734628,True,heuristique_malplace,7.671043,0.0 minutes et 7.6710429191589355 secondes,4965,True


In [ ]:
!pip install dataframe-image

     ---------------------------------------- 6.6/6.6 MB 2.7 MB/s eta 0:00:00


In [ ]:
import dataframe_image as dfi
dfi.export(df.style, 'df_style.png')

In [ ]:
!pip install pyflowchart